In [1]:
import os
import numpy as np
import shutil
from glob import glob
import librosa
from IPython import display
#import cPickle as pickle
from pysndfx import AudioEffectsChain

In [ ]:
#Rename speaker folders in numeric order to represent ID.
speakers = glob("data/LibriSpeech-360/train-clean-360/*")
for speaker in speakers:                                    #Make sure only valid speaker directories are in list.
    assert os.path.isdir(speaker)
    assert str.isdigit(os.path.split(speaker)[1])
print(len(speakers))
for i in range(0,len(speakers)):
    old_path = speakers[i] 
    new_path = os.path.split(speakers[i])[0] + '/' + str(i) 
    os.rename(old_path, new_path)

In [ ]:
#Rearrange files in dataset LibriSpeech folder so that all audios of a particular speaker are in the same directory.

speakers = glob("data/LibriSpeech-360/train-clean-360/*/")
for speaker in speakers:
    subdirs = os.listdir(speaker)                         #List subdirectories for a particular speaker.
    for subdir in subdirs:
        if os.path.isdir(speaker + subdir):               #Check if its a directory and not a file.
            audio_files = os.listdir(speaker + subdir)    #List files for a particular speaker in a particular subfolder.
            for audio_file in audio_files:
                if audio_file.endswith('.flac'):          #Check if file is audio file and not a text file or something unwanted.
                    old_path = speaker + subdir + '/' + audio_file 
                    new_path = speaker + audio_file
                    os.rename(old_path, new_path)         #Move the audiofile to main speaker directory.
            shutil.rmtree(speaker + subdir)               #Remove subdirectory.

In [52]:
#Delete audios less than 4 sec.
no_audios = 0
small_audios = []
audio_files = glob("data/evaluation_normalized/*/*.wav")
for audio_file in audio_files:
    audio_data, audio_rate = librosa.load(sr=500, mono=True, path=audio_file)
    no_audios += 1
    if (no_audios % 100 == 0):
        print(no_audios)
    if len(audio_data) < (audio_rate*4):
        small_audios.append(audio_file)
print('\n-----------------------------------------------------------------------')
print(len(small_audios))
#for audio_file in small_audios:
#    os.remove(audio_file)


-----------------------------------------------------------------------
0


In [51]:
#Append zero to audios less than 4 sec
for audio_file in audio_files:
    audio_data, audio_rate = librosa.load(sr=16000, mono=True, path=audio_file)
    if len(audio_data) >= (audio_rate*4):
        librosa.output.write_wav(audio_file, audio_data, audio_rate, norm=False)
    elif len(audio_data) < (audio_rate*4):
        length_diff = (audio_rate*4) - len(audio_data)
        zeroes = [0 for i in range(0,length_diff)]
        new_audio = np.append(audio_data, zeroes)
        librosa.output.write_wav(audio_file, new_audio, audio_rate, norm=False)